# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [45]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [46]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,6.05,96,100,7.63,TF,1702352501
1,1,kapenguria,1.2511,35.0895,14.23,81,88,1.57,KE,1702352501
2,2,muisne,0.6000,-80.0333,25.22,89,61,2.83,EC,1702352501
3,3,thompson,55.7435,-97.8558,-14.91,72,40,6.69,CA,1702352208
4,4,blackmans bay,-43.0167,147.3167,23.62,45,100,3.53,AU,1702352501


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [47]:
%%capture --no-display

# Configure the map plot
city_humidity_map = city_data_df.hvplot.points("Lng", "Lat", xlabel="x", ylabel="y", xlim=[-180,180],
                                                color="City", size="Humidity", geo=True, tiles = "OSM")
# Display the map
city_humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [48]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) 
                                 & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
ideal_city_data_df = ideal_city_data_df.dropna()

# Display sample data
ideal_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
134,134,veinticinco de mayo,-35.4323,-60.1727,21.10,80,0,0.89,AR,1702352526
215,215,vila velha,-20.3297,-40.2925,26.08,81,0,4.02,BR,1702352410
401,401,teknaf,20.8624,92.3058,23.42,66,0,1.70,BD,1702352575
413,413,navrongo,10.8956,-1.0921,22.19,21,0,4.10,GH,1702352577
459,459,bhamo,24.2667,97.2333,24.37,49,0,1.28,MM,1702352588
497,497,cabo san lucas,22.8909,-109.9124,23.20,76,0,2.57,MX,1702352140
565,565,salalah,17.0151,54.0924,22.05,28,0,4.12,OM,1702352617


### Step 3: Create a new DataFrame called `hotel_df`.

In [49]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = "" #set each entry in new column blank

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
134,veinticinco de mayo,AR,-35.4323,-60.1727,80,
215,vila velha,BR,-20.3297,-40.2925,81,
401,teknaf,BD,20.8624,92.3058,66,
413,navrongo,GH,10.8956,-1.0921,21,
459,bhamo,MM,24.2667,97.2333,49,
497,cabo san lucas,MX,22.8909,-109.9124,76,
565,salalah,OM,17.0151,54.0924,28,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [50]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey" : geoapify_key,
    "categories": "accommodation.hotel" 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
veinticinco de mayo - nearest hotel: Jorly
vila velha - nearest hotel: Hotel Prainha
teknaf - nearest hotel: No hotel found
navrongo - nearest hotel: Mayaga
bhamo - nearest hotel: Winlight Hotel
cabo san lucas - nearest hotel: Comfort Rooms
salalah - nearest hotel: Muscat International Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
134,veinticinco de mayo,AR,-35.4323,-60.1727,80,Jorly
215,vila velha,BR,-20.3297,-40.2925,81,Hotel Prainha
401,teknaf,BD,20.8624,92.3058,66,No hotel found
413,navrongo,GH,10.8956,-1.0921,21,Mayaga
459,bhamo,MM,24.2667,97.2333,49,Winlight Hotel
497,cabo san lucas,MX,22.8909,-109.9124,76,Comfort Rooms
565,salalah,OM,17.0151,54.0924,28,Muscat International Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [51]:
%%capture --no-display

# Configure the map plot
hotels_map = hotel_df.hvplot.points("Lng", "Lat", hover_cols = ["Hotel Name", "Country"],
                                    xlabel="x", ylabel="y", xlim=[-180,180],
                                    color="City", size="Humidity", 
                                    geo=True, tiles = "OSM", frame_width = 600, frame_height = 400 #need to set due to greatly expanded image
)

# Display the map
hotels_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)